In [ ]:
# default_exp hypothesis_space
%load_ext lab_black # nb_black if running in jupyter
%load_ext autoreload # automatically reload python modules if there are changes in the 
%autoreload 2

# Hypotheses Space

***input***: toy dataset from data-notebook

***output***: ML model / simulation / analytics python module

***description:***

A hypotheses space defines the possible machine learning models, simulations or analytics tools applied to your problem. 
In this notebook you define the hypotheses space. If you are doing anyhting more complicated than just fitting existing well defined models such as defined in sklearn,
it is recommended that you create a base class for the whole hypotheses space where you define core function handles,
and implement the functions in subclasses that inherit from the base class. If the methods are complicated or you are 
comparing multiple methods that are inheritantly different by nature, you can separate models or subclasses to different notebooks similar to this.
Adjust the running number in the beginning of the notebook name to your needs for retaining logical order.
You should also unit test the classes created in this notebook with either toy data or small sample of your training data.
Remember to add `# export` to top of all cells containing functions or classes that you have defined and want to use outside this notebook.



In [ ]:
#hide
from nbdev.showdoc import *

## Import relevant modules

## Import toy data for testing

## Define base class for hypotheses space / simulations / analytics

## Define subclasses & functions

you should also define the loss function used for model fitting

don't forget to unit test!

## Visualize model behaviour with toy data

## Output of this notebook

The result of this notebook is a collection methods ready for evaluation with the real data.

the methods should be exported with `nbdev_build_lib`, but in the future this will be automatically handled by the pipeline